In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [2]:
proj = np.array([[1, 0], [0, 0]])

# Selecting the device
dev = qml.device("default.mixed", wires=3)

# Function to create the CHSH observables
def create_observable(theta, phi):
    return np.array([[np.cos(theta), np.exp(-1j * phi) * np.sin(theta)],
                     [np.exp(1j * phi) * np.sin(theta), -np.cos(theta)]])

In [3]:
def bell_pair(params,p):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.U3(params[0],params[1],params[2],wires=1)
    qml.U3(params[3],params[4],params[5],wires=2)
    qml.CNOT(wires=[2,1])
    qml.RZ(params[6],wires=1)
    qml.RY(params[7],wires=2)
    qml.CNOT(wires=[1,2])
    qml.RY(params[8],wires=2)
    qml.CNOT(wires=[2,1])
    qml.U3(params[9],params[10],params[11],wires=1)
    qml.U3(params[12],params[13],params[14],wires=2)
    qml.DepolarizingChannel(1-p,wires=1)
    qml.DepolarizingChannel(1-p,wires=2)
    qml.adjoint(qml.U3(params[12],params[13],params[14],wires=2))
    qml.adjoint(qml.U3(params[9],params[10],params[11],wires=1))
    qml.CNOT(wires=[2,1])
    qml.adjoint(qml.RY(params[8],wires=2))
    qml.CNOT(wires=[1,2])
    qml.adjoint(qml.RY(params[7],wires=2))
    qml.adjoint(qml.RZ(params[6],wires=1))
    qml.CNOT(wires=[2,1])
    qml.adjoint(qml.U3(params[3],params[4],params[5],wires=2))
    qml.adjoint(qml.U3(params[0],params[1],params[2],wires=1))

    qml.QubitUnitary(proj, wires=2)



In [4]:
@qml.qnode(dev)
def bell_pair1(params,p):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    qml.U3(params[0],params[1],params[2],wires=1)
    qml.U3(params[3],params[4],params[5],wires=2)
    qml.CNOT(wires=[2,1])
    qml.RZ(params[6],wires=1)
    qml.RY(params[7],wires=2)
    qml.CNOT(wires=[1,2])
    qml.RY(params[8],wires=2)
    qml.CNOT(wires=[2,1])
    qml.U3(params[9],params[10],params[11],wires=1)
    qml.U3(params[12],params[13],params[14],wires=2)
    qml.DepolarizingChannel(1-p,wires=1)
    qml.DepolarizingChannel(1-p,wires=2)
    qml.adjoint(qml.U3(params[12],params[13],params[14],wires=2))
    qml.adjoint(qml.U3(params[9],params[10],params[11],wires=1))
    qml.CNOT(wires=[2,1])
    qml.adjoint(qml.RY(params[8],wires=2))
    qml.CNOT(wires=[1,2])
    qml.adjoint(qml.RY(params[7],wires=2))
    qml.adjoint(qml.RZ(params[6],wires=1))
    qml.CNOT(wires=[2,1])
    qml.adjoint(qml.U3(params[3],params[4],params[5],wires=2))
    qml.adjoint(qml.U3(params[0],params[1],params[2],wires=1))

    qml.QubitUnitary(proj, wires=2)

    return qml.density_matrix(wires=[0,1])

In [5]:
def measure_AB(dev, bell_pair, A, B):
    @qml.qnode(dev)
    def circuit(params,p):
        bell_pair(params,p) 
        return qml.expval(qml.Hermitian(A, wires=0) @ qml.Hermitian(B, wires=1))
    return circuit

# Cost function
def cost(params,p):
    # Creating observables with new parameters
    A1 = create_observable(0, 0)
    A2 = create_observable(np.pi/2, 0)
    B1 = create_observable(np.pi/4, 0)
    B2 = create_observable(np.pi/4, np.pi)
    trace=np.trace(bell_pair1(params,p))

    expvals = [measure_AB(dev, bell_pair, A1, B1)(params,p)/trace,
               measure_AB(dev, bell_pair, A1, B2)(params,p)/trace,
               measure_AB(dev, bell_pair, A2, B1)(params,p)/trace,
               measure_AB(dev, bell_pair, A2, B2)(params,p)/trace]
    # trace=1 #np.trace(bell_pair1(params,p))
    return -np.real((np.sum(expvals[:3]) - expvals[3]))

In [6]:
p_val =np.linspace(0.25,1,0.5)

CHSH = []
par = []
Prob_SUCESS = []
nstep = 500

adam_opt = qml.AdamOptimizer(0.1)
adam_steps = 100 

for p in p_val:
    # Initialize parameters
    params = np.array(np.random.uniform(-np.pi, np.pi, 15), requires_grad=True)
    

    for i in range(adam_steps):
        params, ccost = adam_opt.step_and_cost(lambda v: cost(v, p), params)
        print(f"step {i + 1}/{adam_steps}, Cost: {ccost}")

    # Calculate the success probability
    PS = np.trace(bell_pair1(params, p))

    # Store the result for this p_val
    CHSH.append(ccost)
    par.append(params)
    Prob_SUCESS.append(PS)

# Output the results
print("CHSH:", CHSH)
print("Params:", par)
print("Prob_SUCESS:", Prob_SUCESS)

Adam Fine-Tuning Step 1/100, Cost: -0.7712375860117111
Adam Fine-Tuning Step 2/100, Cost: -0.8517952095154528
Adam Fine-Tuning Step 3/100, Cost: -0.8626162610657673
Adam Fine-Tuning Step 4/100, Cost: -0.8473495718356443
Adam Fine-Tuning Step 5/100, Cost: -0.8489539063050713
Adam Fine-Tuning Step 6/100, Cost: -0.8678356088189593
Adam Fine-Tuning Step 7/100, Cost: -0.8862626179773551
Adam Fine-Tuning Step 8/100, Cost: -0.8904080679813979
Adam Fine-Tuning Step 9/100, Cost: -0.8816161214944607
Adam Fine-Tuning Step 10/100, Cost: -0.8726900196865341
Adam Fine-Tuning Step 11/100, Cost: -0.871975444876375
Adam Fine-Tuning Step 12/100, Cost: -0.8781474173742017
Adam Fine-Tuning Step 13/100, Cost: -0.8852936532005007
Adam Fine-Tuning Step 14/100, Cost: -0.8883574851556006
Adam Fine-Tuning Step 15/100, Cost: -0.886749168230897
Adam Fine-Tuning Step 16/100, Cost: -0.8841299871176536
Adam Fine-Tuning Step 17/100, Cost: -0.884083161922867
Adam Fine-Tuning Step 18/100, Cost: -0.8867591769713129
Adam

In [7]:
# [tensor(1.99699575, requires_grad=True), tensor(2.00598162, requires_grad=True), tensor(2.02460369, requires_grad=True), tensor(2.0581806, requires_grad=True), tensor(2.11178293, requires_grad=True), tensor(2.18644232, requires_grad=True), tensor(2.28227571, requires_grad=True), tensor(2.39369046, requires_grad=True), tensor(2.51868766, requires_grad=True), tensor(2.65319462, requires_grad=True), tensor(2.79724831, requires_grad=True)]

In [8]:
f1_anat = [1.1313708498984758,1.365447577463678,1.5896999168281507,1.8007978925719688,1.9965367939384875,2.1757131728816845,2.3379484325936715,2.4834969875820208,2.613064653318815,2.7276518115129527,2.8284271247461903]
f1num=[1.26277036,1.48416195,1.69116265,1.8823018,2.05786114,2.21824051,2.36447021,2.49840787,2.61949019,2.69858243,2.82842555]
p1num=[0.5,0.5801684047848221,0.6089193863872889,0.6422274086134266,0.6800251336292099,0.7223578403515133,0.7691523517309242,0.8200857682076388,0.8756233266968312,0.9447131992968434,1.0000000000000533]

In [9]:
ff1num=[1.131370849875084,1.3654475771792844,1.5896999166948438,1.800797892502143,1.996536293428982,2.1757103639895634,2.3379472097006033,2.48349622334479,2.6130640854024603,2.727651435832109,2.8284271247461215]
fp1num=[0.555555555570983,0.580000000067194,0.6088888889081859,0.6422222222258873,0.6800001208192743,0.7222226034023373,0.7688892860072644,0.8200004643841046,0.8755557296409249,0.9355556033845127,1.0000000000000426]